In [40]:
# İSTER 1
from pymongo import MongoClient
from collections import deque
from time import perf_counter

start = perf_counter()

client = MongoClient("mongodb://localhost:27017")
db = client["mongodb_relations"]
relations = db["relations"]
persons = db["persons"]

source_name = "Gülnur Yıldız"
target_name = "Ahmet Doğan"
depth = 3
counter = 0

queue = deque()
queue.append((source_name, [(source_name, None)])) 

visited = set()

while queue:
    current, path = queue.popleft()

    if len(path) < depth + 1:
        pass
    elif len(path) == depth + 1:
        if current == target_name:
            yol_str = ""
            for i in range(len(path) - 1):
                yol_str += path[i][0]
                if path[i + 1][1]:
                    yol_str += f" -({path[i + 1][1]})-> "
            yol_str += path[-1][0] 
            print(yol_str)
        continue
    else:
        continue

    if current in visited:
        continue
    visited.add(current)

    neighbors = relations.find({ "$or": [ {"source": current}, {"target": current} ] })

    for rel in neighbors:
        next_person = rel["target"] if rel["source"] == current else rel["source"]
        if next_person not in [x[0] for x in path]:
            relation_types = rel.get("relations", [])
            relation_label = relation_types[0]
            queue.append((next_person, path + [(next_person, relation_label)]))

end = perf_counter()
print(f"{end - start:.6f} saniye")


Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(AYNI_MEMLEKETTEN)-> Alperen Yılmaz -(LISE_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(AYNI_MEMLEKETTEN)-> Ayhan Korkmaz -(UNIVERSITE_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(ILGI_ALANI_ORTAKLIGI)-> Ayşegül Keskin -(UNIVERSITE_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(ILGI_ALANI_ORTAKLIGI)-> Bahar Erdoğan -(AYNI_MEMLEKETTEN)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(ILGI_ALANI_ORTAKLIGI)-> Batuhan Özdemir -(LISE_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(ILGI_ALANI_ORTAKLIGI)-> Bülent Yıldız -(AYNI_MEMLEKETTEN)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(ILGI_ALANI_ORTAKLIGI)-> Burak Acar -(YUKSEK_LISANS_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Çakır -(AYNI_MEMLEKETTEN)-> Burcu Özsoy -(LISE_ARKADASI)-> Ahmet Doğan
Gülnur Yıldız -(DOKTORA_ARKADASI)-> Ayla Ç

In [41]:
# İSTER 2
from pymongo import MongoClient
from time import perf_counter

start = perf_counter()

client = MongoClient("mongodb://localhost:27017")
db = client["mongodb_relations"]
relations = db["relations"]
persons = db["persons"]

source_name = "Nefise Aydın"

kaynak = persons.find_one({"ad": source_name})
ilkokul = kaynak.get("ilkokul")

ara_kisiler = relations.find({
    "source": source_name,
    "relations": { "$in": ["ILKOKUL_ARKADASI"] }
})

ara_listesi = [rel["target"] for rel in ara_kisiler]

ara_kisiler_bilgileri = persons.find({"ad": {"$in": ara_listesi}})
universiteye_gore_ara = {}
for kisi in ara_kisiler_bilgileri:
    univ = kisi.get("universite")
    if univ:
        universiteye_gore_ara.setdefault(univ, []).append(kisi["ad"])

hedef_kisiler = []
for univ, isimler in universiteye_gore_ara.items():
    ayni_unide_olanlar = persons.find({
        "universite": univ,
        "ad": {"$nin": isimler} 
    })
    hedef_kisiler.extend([p["ad"] for p in ayni_unide_olanlar])

print("Hedef kişiler:")
for isim in hedef_kisiler:
    print("-", isim)

print(len(hedef_kisiler))

end = perf_counter()
print(f"Çalışma süresi: {end - start:.4f} saniye")


Hedef kişiler:
- Akın Doğan
- Betül Erdoğan
- Burak Acar
- Burhan Sezer
- Cem Şimşek
- Ercan Yılmaz
- Erhan Keskin
- Fatma Özkan
- Fikri Bulut
- Gülnur Toprak
- Halit Baran
- Haluk Bulut
- Haluk Erdoğan
- Hasan Aksoy
- Hümeyra Güler
- Hüsnü Aktaş
- İkra Demir
- İpek Aksoy
- İsmail Uçar
- İzzet Özkan
- İzzet Yıldız
- Kemal Can
- Melek Ermiş
- Mert Öztürk
- Necati Aslan
- Onur Aslan
- Ozan Aslan
- Özgür Şimşek
- Ramazan Kaplan
- Salih Demir
- Sedef Sezer
- Serap Arslan
- Sevda Toprak
- Şirin Erdoğan
- Süleyman Eren
- Süleyman Kara
- Tarık Aktaş
- Vedat Tan
- Veli Taş
- Yunus Karaca
- Zafer Bozkurt
- Ahmet Doğan
- Ali Türkmen
- Alperen Yılmaz
- Ayhan Korkmaz
- Ayşegül Keskin
- Batuhan Özdemir
- Baturalp Polat
- Bülent Yıldız
- Burhan Doğan
- Ceyda Doğan
- Duygu Ergin
- Eda Altun
- Ferhat Kurt
- Fuat Bulut
- Gökhan Tan
- Hakan Şimşek
- Hasan Bozkurt
- Hatice Polat
- Hüsnü Kurt
- İskender Kara
- İsmail Eren
- İzzet Arslan
- Kaan Güneş
- Kaan Özdemir
- Kadir Altun
- Levent Ermiş
- Mahir Koç


In [42]:
# İSTER 3
from pymongo import MongoClient
from time import perf_counter

start = perf_counter()

client = MongoClient("mongodb://localhost:27017")
db = client["mongodb_relations"]
relations = db["relations"]
persons = db["persons"]

isim = "İzzet Yıldız"

kaynak = persons.find_one({"ad": isim})
dogum_yeri = kaynak.get("dogum_yeri")

related_docs = relations.find({
    "$or": [
        {"source": isim},
        {"target": isim}
    ]
})

hedef_adlari = set()
for rel in related_docs:
    if rel["source"] == isim:
        hedef_adlari.add(rel["target"])
    else:
        hedef_adlari.add(rel["source"])

hemsehriler = persons.find({
    "ad": {"$in": list(hedef_adlari)},
    "dogum_yeri": dogum_yeri
})

print("Hemşehrilerin ve çalıştıkları son kurum:")
for kisi in hemsehriler:
    kurumlar = kisi.get("calistigi_kurumlar", [])
    son_kurum = kurumlar[-1] if kurumlar else "(bilgi yok)"
    print(f"- {kisi['ad']} → {son_kurum}")

end = perf_counter()
print(f"\nÇalışma süresi: {end - start:.4f} saniye")


Hemşehrilerin ve çalıştıkları son kurum:
- Ahmet Doğan → Artemis Yazılım
- Alperen Baran → Garanti BBVA
- Alperen Uzun → Koç Üniversitesi Vehbi Koç Ankara Araştırmaları Uygulama ve Araştırma Merkezi (VEKAM)
- Bahar Erdoğan → Anadolu Mirası Sürdürülebilir Turizm Derneği
- Bülent Yıldız → VeriVizyon AI
- Ebru Avcı → "Yeşil Adımlar" Sürdürülebilir Kent Danışmanlığı
- Emrah Baran → Sahibinden.com
- Faruk Korkmaz → EKO Kent Tasarım Atölyesi
- Gülnur Toprak → Microsoft Türkiye
- Halit Aktaş → Anka Prodüksiyon
- Haluk Bulut → Yıldız Holding
- Haluk Erdoğan → Yeşil Gelecek Danışmanlık
- Harun Koç → Miras ve Kent Danışmanlık A.Ş.
- Hasan Aksoy → UrbanSense Teknoloji A.Ş.
- İpek Aksoy → İzmir Ekonomi Üniversitesi Sosyoloji Bölümü
- İsmail Eren → İnsan Verisi Lab
- İzzet Özkan → Paraşüt
- İzzet Yalçın → Vizyon Dijital Ajans
- Kaan Yılmaz → McCann Istanbul
- Kemal Can → Getir
- Melek Ermiş → Microsoft Türkiye
- Mert Öztürk → Getir
- Necla Güneş → UrbaniX Consult
- Nihat Şimşek → Sabancı Üniversite

In [43]:
# İSTER 4
from pymongo import MongoClient
from time import perf_counter

start = perf_counter()

client = MongoClient("mongodb://localhost:27017")
db = client["mongodb_relations"]
relations = db["relations"]
persons = db["persons"]

source_name = "Emrah Baran"

# 1. Emrah Baran'ın yüksek lisans bilgisini al
kaynak = persons.find_one({"ad": source_name})
if not kaynak:
    print("Kişi bulunamadı.")
    exit()

yuksek_lisans = kaynak.get("yuksek_lisans")
if not yuksek_lisans:
    print("Yüksek lisans bilgisi yok.")
    exit()

# 2. Emrah Baran'a bağlı tüm kişiler (tek yönlü)
baglantilar = relations.find({"source": source_name})

# 3. Bağlantılı kişilerin adlarını topla
hedef_adlari = [rel["target"] for rel in baglantilar]

# 4. Aynı yüksek lisansa sahip olanları filtrele
hedefler = persons.find({
    "ad": {"$in": hedef_adlari},
    "yuksek_lisans": yuksek_lisans
})

# 5. Sonuçları yazdır
print("Yüksek lisans arkadaşlarının çocukları:")
for kisi in hedefler:
    print(f"- {kisi['ad']} → {kisi.get('cocuklar', [])}")

end = perf_counter()
print(f"\nÇalışma süresi: {end - start:.4f} saniye")

Yüksek lisans arkadaşlarının çocukları:
- Engin Tan → ['Can', 'Elif']
- Ferhat Kurt → ['Elif Kurt', 'Can Kurt']
- Gökhan Toprak → ['Ela Toprak', 'Can Toprak']
- Gülay Sezer → ['Ege Sezer', 'Defne Sezer']
- Gülnur Toprak → ['Lara']
- Harun Taş → ['Ela Taş', 'Can Taş']
- İrem Yalçın → ['Aslı', 'Deniz']
- İsmail Aktaş → ['Elif Aktaş', 'Can Aktaş']
- Kaan Güneş → ['Defne Güneş', 'Can Güneş']
- Meltem Altun → ['Defne']
- Nazan Türkmen → ['Elif']
- Necati Öztürk → ['Mert Öztürk', 'Elif Öztürk']
- Neşe Demir → ['Ada', 'Efe']
- Ozan Aslan → ['Deniz', 'Emir']
- Reşat Şimşek → ['Ada', 'Can']
- Selin Acar → ['Ela Acar Yılmaz', 'Demir Acar Yılmaz']
- Semra Eren → ['Ada Demir', 'Can Demir']
- Serap Arslan → ['Elif Aydemir', 'Bora Aydemir']
- Şirin Altun → ['Ada', 'Can']
- Şirin Erdoğan → ['Ada', 'Ege']
- Süleyman Eren → ['Defne', 'Emre']
- Veli Taş → ['Deniz', 'Can']
- Vildan Güler → ['Elif Yılmaz', 'Can Yılmaz']
- Zeynep Avcı → ['Ela', 'Can']
- Zeynep Uzun → ['Ada', 'Bora']

Çalışma süresi: 0.0515

In [44]:
# İSTER 5
from pymongo import MongoClient
from time import perf_counter

start = perf_counter()

client = MongoClient("mongodb://localhost:27017")
db = client["mongodb_relations"]
relations = db["relations"]
persons = db["persons"]

source_name = "Ali Türkmen"

# 1. Ali'nin bilgilerini al
kaynak = persons.find_one({"ad": source_name})
if not kaynak:
    print("Kişi bulunamadı.")
    exit()

kaynak_kurumlar = kaynak.get("calistigi_kurumlar", [])
kaynak_yillar = kaynak.get("calisma_yillari", [])

kaynak_kurum_yillar = list(zip(kaynak_kurumlar, kaynak_yillar))

# 2. Ali ile ilişkili tüm kişileri bul (yönsüz)
related_docs = relations.find({
    "$or": [
        {"source": source_name},
        {"target": source_name}
    ]
})

# 3. Karşıdaki kişi adlarını topla
hedef_adlari = set()
for rel in related_docs:
    other = rel["target"] if rel["source"] == source_name else rel["source"]
    hedef_adlari.add(other)

# 4. Kişi bilgilerini getir
hedefler = persons.find({"ad": {"$in": list(hedef_adlari)}})

# 5. Ortak kurum + yıl kontrolü
sonuclar = []

def parse_years(year_str):
    try:
        start, end = map(int, year_str.split("-"))
        return start, end
    except:
        return None, None

for hedef in hedefler:
    hedef_kurumlar = hedef.get("calistigi_kurumlar", [])
    hedef_yillar = hedef.get("calisma_yillari", [])
    hedef_kurum_yillar = list(zip(hedef_kurumlar, hedef_yillar))

    for kurum1, yil1 in kaynak_kurum_yillar:
        b1, e1 = parse_years(yil1)
        if b1 is None: continue

        for kurum2, yil2 in hedef_kurum_yillar:
            if kurum1 != kurum2: continue

            b2, e2 = parse_years(yil2)
            if b2 is None: continue

            ortak_bas = max(b1, b2)
            ortak_bit = min(e1, e2)

            if ortak_bit >= ortak_bas:
                birlikte_yil = ortak_bit - ortak_bas + 1
                sonuclar.append({
                    "İsim": hedef["ad"],
                    "Yaşadığı_Şehir": hedef.get("yasadigi_sehir", "Bilinmiyor"),
                    "Ortak_Kurum": kurum1,
                    "Birlikte_Yıl": birlikte_yil
                })

# 6. Sonuçları sırala ve yazdır
sonuclar.sort(key=lambda x: x["Birlikte_Yıl"], reverse=True)

for sonuc in sonuclar:
    print(f"{sonuc['İsim']} | {sonuc['Yaşadığı_Şehir']} | {sonuc['Ortak_Kurum']} | {sonuc['Birlikte_Yıl']} yıl")

end = perf_counter()
print(f"\nÇalışma süresi: {end - start:.4f} saniye")


Ahmet Doğan | İstanbul | ASELSAN | 4 yıl
Akın Doğan | Ankara | ASELSAN | 4 yıl
Burak Acar | İstanbul | ASELSAN | 4 yıl
Burhan Doğan | İstanbul | ASELSAN | 4 yıl
Gülnur Toprak | İstanbul | ASELSAN | 4 yıl
Hüsnü Aktaş | Ankara | ASELSAN | 4 yıl
Hüsnü Kurt | İzmir | ASELSAN | 4 yıl
İrem Yalçın | İstanbul | TEMA Vakfı | 4 yıl
Kaan Güneş | İstanbul | ASELSAN | 4 yıl
Necati Aslan | İstanbul | ASELSAN | 4 yıl
Özge Karaca | Ankara | ASELSAN | 4 yıl
Süleyman Eren | İstanbul | ASELSAN | 4 yıl
Tarık Aktaş | Ankara | ASELSAN | 4 yıl
Yunus Karaca | Ankara | ASELSAN | 4 yıl
Zehra Ermiş | İstanbul | ASELSAN | 3 yıl
Levent Ermiş | İzmir | Ankara Büyükşehir Belediyesi | 2 yıl
Sevda Toprak | İstanbul | Ankara Büyükşehir Belediyesi | 2 yıl
Şirin Bozkurt | İstanbul | Birleşmiş Milletler Kalkınma Programı (UNDP) Türkiye Ofisi | 2 yıl
Zeynep Uzun | İstanbul | ASELSAN | 2 yıl
Onur Aslan | Ankara | Ankara Büyükşehir Belediyesi | 1 yıl
Yunus Karaca | Ankara | ASELSAN | 1 yıl

Çalışma süresi: 0.0839 saniye
